# 中間層の取得
参考URL: https://note.nkmk.me/python-tensorflow-keras-applications-pretrained-models/

In [33]:
import numpy as np
import os
from tqdm import tqdm
from tensorflow.keras.applications.efficientnet import EfficientNetB7, preprocess_input
from tensorflow.keras.preprocessing import image

## input_shape:
入力する画像のshapeを指定する, (入力画像数, height, width, チャンネル数)となる。  
1枚だけの処理でも入力画像数を指定しなければいけないので, np.expand_dims()などを用いる。

## include_top:
出力時に画像をクラスに分けるかどうか, imagenetで訓練されたモデルなら1000クラスに分ける。  
Falseにすれば通常だと全結合層以降が除かれる。

## pooling:
モデルの最後にプーリング層をつけるかどうか, avgならGlobalAveragePooling2D層が追加される。

In [2]:
model=EfficientNetB7(weights="imagenet",include_top=False,pooling="avg")

## preprocess_input()
用いるモデルに合わせた前処理を行う

In [36]:
dataset_dir="../input/ranzcr-256x256-dataset/"
dest_dir="../input/efficientnet_output/"

for file_name in tqdm(os.listdir(dataset_dir)):
    img_pil=image.load_img(dataset_dir+file_name) # PIL形式で画像が読み込まれる
    img=image.img_to_array(img_pil)
    img=np.expand_dims(img,axis=0)

    pred=model.predict(preprocess_input(img))
    np.save(dest_dir+os.path.splitext(file_name)[0]+".npy",pred)

100%|██████████| 30083/30083 [13:41:57<00:00,  1.64s/it]
